# 2.1 Amazon SageMaker Training 실습

### 작업 실행 시 필요 라이브러리 import

In [ ]:
import boto3
import sagemaker

### SageMaker 세션과 Role, 사용 버킷 정의

In [ ]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

In [ ]:
# bucket = sagemaker_session.default_bucket()
# code_location = f's3://{bucket}/xgboost/code'
# output_path = f's3://{bucket}/xgboost/output'

### 하이퍼파라미터 정의

In [ ]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### 학습 실행 작업 정의

In [ ]:
instance_count = 1
instance_type = "ml.m5.large"
# instance_type = "local"
# instance_type = "local_gpu"

max_run = 1*60*60

use_spot_instances = False
if use_spot_instances:
    max_wait = 1*60*60
else:
    max_wait = None

In [ ]:
if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()

## FSx for Luster 데이터 채널 준비

In [ ]:
from sagemaker.inputs import FileSystemInput

# Specify FSx Lustre file system id.
# file_system_id = # "fs-xxxxxxxxxxxxxx"
file_system_id = 'fs-092cbdbab4de3f550'

# Specify directory path for input data on the file system. 
# You need to provide normalized and absolute path below.
# base_path = "<your-mount-name>"

###########################
# Train Data
###########################
base_path = '/qbryzbev'

file_system_directory_path = f'{base_path}/fraud'
print(f'FSx file-system data input path: {file_system_directory_path}')


file_system_access_mode = 'rw'

train = FileSystemInput(file_system_id=file_system_id,
                        file_system_type=file_system_type,
                        directory_path=file_system_directory_path,
                        file_system_access_mode=file_system_access_mode)

###########################
# Log Data
###########################

log_file_system_directory_path = f'{base_path}/fraud/log'
print(f'FSx file-system log path: {log_file_system_directory_path}')


log = FileSystemInput(file_system_id=file_system_id,
                      file_system_type=file_system_type,
                      directory_path=log_file_system_directory_path,
                      file_system_access_mode=file_system_access_mode)

inputs = {'train': train, 'log': log}


print("inputs: \n", inputs)



## Setup FSx for Luster Network

In [ ]:
# Give Amazon SageMaker Training Jobs Access to FileSystem Resources in Your Amazon VPC.
# security_group_ids = # ['sg-xxxxxxxx'] 
# subnets = # [ 'subnet-xxxxxxx']

subnet_id = 'subnet-f1067697'
security_group_id = 'sg-01d2d82fa2227f449'

security_group_ids = list()
security_group_ids.append(security_group_id)
subnets = list()
subnets.append(subnet_id)
print("security_group_ids: ", security_group_ids)
print("subnets: ", subnets)


## S3 output location

In [ ]:
s3_bucket = 'cf-mask-rcnn-gsmoon'
prefix = "fraud/output"
s3_output_location = f"s3://{s3_bucket}/{prefix}"
print("s3_output_location: ", s3_output_location)

In [ ]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_fsx_luster_script.py",
    source_dir='src',
#    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.3-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    output_path=s3_output_location,    
    max_wait=max_wait,
    keep_alive_period_in_seconds=1800,        
    subnets=subnets,
    security_group_ids=security_group_ids,    
)

In [ ]:
inputs

### 학습 실행

In [ ]:
estimator.fit(inputs = inputs,
                  wait=False)

In [ ]:
estimator.logs()